In [8]:
import os
import csv
import datetime
from datetime import datetime
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import itertools
import warnings

warnings.simplefilter("ignore", UserWarning)

device_list = ['television', 'fan', 'fridge', 'laptop computer', 'electric heating element', 'oven', 'unknown', 'washing machine', 'microwave', 'toaster', 'sockets', 'cooker']
device_dict = {2: 'Television', 3: 'Fan', 4: 'Fridge', 5: 'Laptop computer', 6: 'Electric heating element', 7: 'Oven', 8: 'Unknown', 9: 'Washing machine', 10: 'Microwave', 11: 'Toaster', 12: 'Sockets', 13: 'Cooker'}




In [9]:
def process_file(filename, device_dict):
    # Initialize power_dict with keys for each device name (lowercase)
    power_dict = {key.lower(): {} for key in device_dict.values()}

    # Get file name and create date object
    date_str = os.path.basename(filename).split(".")[0]
    samples = []

    # Open file and read through it, adding data to power_dict
    with open(filename, 'r') as file:
        csvreader = csv.reader(file)
        lines = [line for line in csvreader]
        print(len(lines))
        row_count = 26
        current_day = None
        for row in tqdm(itertools.islice(lines, 29, 13301960), total=13301960, smoothing=0):
            date_time = datetime.strptime(row[0], '%Y-%m-%d %H:%M:%S%z')
            if current_day is None:
                current_day = date_time.date()
            elif date_time.date() != current_day:
                processed_data = formatting(power_dict)
                samples.append(processed_data)
                power_dict = {key.lower(): {} for key in device_dict.values()}
                current_day = date_time.date()
            count = 0
            for value in row[1:]:
                power_dict[device_list[count]][date_time] = float(value)
                count += 1
    return samples





In [10]:
def formatting(power_dict):
    processed_data = {}
    for appliance in power_dict:
        # Convert the data in the inner dictionary to a dataframe
        power_dict[appliance] = pd.DataFrame.from_dict(power_dict[appliance], orient='index', columns=[appliance])
        power_dict[appliance].index = pd.to_datetime(power_dict[appliance].index, utc=True)
        # Resample the dataframe to 7 seconds intervals and backfill the missing values
        power_dict[appliance] = power_dict[appliance].resample('7s').ffill(limit=1).fillna(0)
        # Convert the index to period of seconds
        power_dict[appliance].index = power_dict[appliance].index.to_period('s')
        processed_data[appliance] = power_dict[appliance]
    return processed_data

In [ ]:
path = 'DRED'
files = os.listdir(path)

samples = process_file('DRED/Appliance_data.csv', device_dict)

processed_data = {key: [sample[key] for sample in samples] for key in device_list}

import pickle
# Save processed data to a pickle file
with open('DRED_processed.pickle', 'wb') as handle:
    pickle.dump(processed_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
